In [ ]:
from Data_Preprocessing import prepare_data
import pandas as pd
import numpy as np
import math
from numpy import nan
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
!rm *.csv
from google.colab import files
print("Please load the raw csv data")
uploadedData = files.upload()
filename = list(uploadedData)[0]
dataset = pd.read_csv(filename)
train_data, test_data= train_test_split(dataset, test_size=0.2, train_size=0.8, random_state=156)

dataset['risk'].replace({1: 1, 0: -1}, inplace=True)
train_data['risk'].replace({1: 1, 0: -1}, inplace=True)
test_data['risk'].replace({1: 1, 0: -1}, inplace=True)

dataset['spread'].replace({1: 1, 0: -1}, inplace=True)
train_data['spread'].replace({1: 1, 0: -1}, inplace=True)
test_data['spread'].replace({1: 1, 0: -1}, inplace=True)

dataset['covid'].replace({1: 1, 0: -1}, inplace=True)
train_data['covid'].replace({1: 1, 0: -1}, inplace=True)
test_data['covid'].replace({1: 1, 0: -1}, inplace=True)

# dataset, train_data = prepare_data(dataset, train_data)
test_data, train_data = prepare_data(test_data, train_data)


In [ ]:
# **Part 2: Decision trees:**
from visualize_clf import *
from prepare import *

# Visualization and basic analysis
# Q7: histplots from major 1
# Q8: correlated features (not labels)
copy = train_data.copy()
copy = copy.drop(['risk', 'spread'], axis =1)
corr = copy.corr()
print(corr['covid'].sort_values(ascending=False))


# Q9: Desicion trees learning
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
clf = DecisionTreeClassifier(max_depth=4)
X = train_data.copy().drop(['risk', 'spread', 'covid'], axis =1)
y = train_data.copy()['covid']
clf.fit(X, y)
score = clf.score(X, y)
print("the training accuracy is: ", score)

plt.figure(figsize=(20,20))
plot_tree(clf, filled=True, feature_names= X.columns, class_names='covid', fontsize=10)
plt.show()


# Q 10: some figures to conclude my point:
sns.displot(train_data.copy()[['PCR_01','covid']], x="PCR_01", hue='covid')

# Q 11:
from sklearn import decomposition, datasets
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate

X = train_data.copy().drop(['risk', 'spread', 'covid'], axis =1)
y = train_data.copy()['covid']
clf = DecisionTreeClassifier()
pipe = Pipeline(steps=[('clf', clf)])

# determining the range of the hyperparameters:
# max_depth: 
max = len(train_data)
min = max
max_result = 0
for depth in list(range(1, max)):
  temp_tree = DecisionTreeClassifier(max_depth=depth)
  scores = cross_validate(temp_tree, X, y, cv=2)
  mean = scores['test_score'].mean()
  if(mean > 0.7 and min == max):
    min = depth
  if(mean > 0.9):
    max_result = depth
    break
print("the requested range for max_depth is: (",min,",",max_result,")")

# min_samples_leaf:
max = len(train_data)
min_2 = max
max_result_2 = 0
for min_leaf in list(range(1, max)):
  temp_tree = DecisionTreeClassifier(min_samples_leaf=min_leaf)
  scores = cross_validate(temp_tree, X, y, cv=2)
  mean = scores['test_score'].mean()
  print(mean, "for: ", min_leaf)
  if(mean > 0.7 and min == max):
    min_2 = min_leaf
  if(mean > 0.9):
    max_result_2 = min_leaf
    break
print("the requested range for min_samples_leaf is: (",min_2,",",max_result_2,")")

max_result = 30
max_result_2 = 210
min_2 = 1
min = 1

#  now tuning both hyperparameters:
depth_range = range(min, max_result)
leaf_range = range(min_2, max_result_2)
parameters = dict(clf__max_depth= depth_range, clf__min_samples_leaf= leaf_range)
clf_GS = GridSearchCV(pipe, parameters, cv=8, return_train_score=True)
clf_GS.fit(X, y)
print('Best max_depth:', clf_GS.best_estimator_.get_params()['clf__max_depth'])
print('Best min_samples_leaf:', clf_GS.best_estimator_.get_params()['clf__min_samples_leaf'])

mean_scores = clf_GS.cv_results_['mean_test_score']
mean_scores = np.array(mean_scores).reshape(len(depth_range),len(leaf_range))
fig, ax = plt.subplots(figsize=(12,12))
ax = sns.heatmap(data= mean_scores, cbar = True, xticklabels = list(leaf_range))
ax.set(xlabel='min_samples_leaf', ylabel='max_depth', title='Mean validation accuracy')

fig2, ax2 = plt.subplots(figsize=(12,12))
mean_train_scores = clf_GS.cv_results_['mean_train_score']
mean_train_scores = np.array(mean_scores).reshape(len(depth_range),len(leaf_range))
ax2 = sns.heatmap(data= mean_train_scores, cbar = True, xticklabels = list(leaf_range))
ax2.set(xlabel='min_samples_leaf', ylabel='max_depth', title='Mean training accuracy')


# Q12: Desicion trees learning
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
clf = DecisionTreeClassifier(max_depth= best_depth, min_samples_leaf= best_leaf)
X = train_data.copy().drop(['risk', 'spread', 'covid'], axis =1)
y = train_data.copy()['covid']
clf.fit(X, y)

test_X = test_data.copy().drop(['risk', 'spread', 'covid'], axis =1)
test_y = test_data.copy()['covid']
score = clf.score(test_X, test_y)
print("the test accuracy is: ", score)

plt.figure(figsize=(20,20))
plot_tree(clf, filled=True, feature_names= X.columns, class_names='covid', fontsize=10)
plt.show()
